In [167]:
%matplotlib inline

# AUTOMATIC DIAGNOSTIC SYSTEM OF SKIN LESIONS FROM DERMOSCOPIC IMAGES


In this practice we are going to build a skin lesion diagnosis system based on dermoscopic image analysis.

## Part 0: The problem

Before starting the practice, we will briefly describe the database that we will use and the problem we aim to address:

Our goal is to develop a CNN providing an automatic diagnosis of cutaneous diseases from dermoscopic images. Dermoscopy is a non-invasive technique that allows the evaluation of the colors and microstructures of the epidermis, the dermoepidermal joint and the papillary dermis that are not visible to the naked eye. These structures are specifically correlated with histological properties of the lesions. Identifying specific visual patterns related to color distribution or dermoscopic structures can help dermatologists decide the malignancy of a pigmented lesion. The use of this technique provides a great help to the experts to support their diagnosis. However, the complexity of its analysis limits its application to experienced clinicians or dermatologists.

In our scenario, we will consider 3 classes of skin lesions:

- Malignant melanoma: Melanoma, also known as malignant melanoma, is the most common type of cancer, and arises from pigmented cells known as melanocytes. Melanomas typically occur on the skin and rarely elsewhere such as the mouth, intestines, or eye.

- Seborrheic keratosis: it is a noncancerous (benign) tumor of the skin that originates from the cells of the outer layer of the skin (keranocytes), so it is a non-melanocytic lesion.

- Benign nevus: a benign skin tumor caused by melanocytes (it is melanocytic)

Figure 1 shows a visual example of the 3 considered lesions:

![Image of ISIC](http://www.tsc.uc3m.es/~igonzalez/images/ISIC.jpg)

The dataset has been obtained from the 'International Skin Imaging Collaboration' (ISIC) file. It contains 2750 images divided into 3 sets:
- Training set: 2000 images
- Validation set: 150 images
- Test set: 600 images

For each clinical case, two images are available:
- The dermoscopic image of the lesion (in the ‘images’ folder).
- A binary mask with the segmentation between injury (mole) and skin (in the 'masks' folder)

Additionally, there is a csv file for each dataset (training, validation and test) in which each lines corresponds with a clinical case, defined with two fields separated by commas:
- the numerical id of the lesion: that allows to build the paths to the image and mask.
- the lesion label: available only for training and validation, being an integer between 0 and 2: 0: benign nevus, 1: malignant melanoma, 2: seborrheic keratosis. In the case of the test set, labels are not available (their value is -1).

Students will be able to use the training and validation sets to build their solutions and finally provide the scores associated with the test set. This practice provides a guideliness to build a baseline reference system. To do so, we will learn two fundamental procedures:

- 1) Process your own database with pytorch
- 2) Fine-tuning a regular network for our diagnostic problem

## Part 1: Handling our custom dataset with pytorch
Now we are going to study how we can load and process our custom dataset in pytorch. For that end, we are going to use the package ``scikit-image`` for reading images, and the package ``panda`` for reading csv files.


In [168]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform, util
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, IterableDataset
from torchvision import transforms, utils, models
import torchvision.datasets as dsets
import torchvision.models.detection as dmodels
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from PIL import Image
import pdb
import random
import numpy.random as npr
from tqdm import tqdm

random.seed(42)
npr.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.enabled = False

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


The first thing we need to do is to download and decompress the dataset on a local folder:




In [150]:
#ONLY TO USE GOOGLE COLAB. Run this code only the first time you run this notebook and then comment these lines
from shutil import copyfile
from google.colab import drive
import os, sys
drive.mount('/content/drive')
copyfile('/content/drive/My Drive/Colab Notebooks/db1.zip', './db1.zip') #Copy db files to our working folder
copyfile('/content/drive/My Drive/Colab Notebooks/db2.zip', './db2.zip')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'./db2.zip'

In [151]:
#NOTE: Run this only once, in the machine where you want to run your code, then comment these lines
import zipfile
zipPath='./db1.zip' #path of the 1st zip file
dataFolder='./data' #We extract files to the current folder
with zipfile.ZipFile(zipPath, 'r') as zip_ref:
    zip_ref.extractall(dataFolder)

zipPath='./db2.zip' #path of the 2nd zip file
dataFolder='./data' # We extract files to the current folder
with zipfile.ZipFile(zipPath, 'r') as zip_ref:
    zip_ref.extractall(dataFolder)

Now let's read the indexed file and display data from image 65. The file structure is one row per image of the database, and two fields:
- Image ID (a 4-digit sequence, adding 0 to the left side if required)
- Label: 0 benign nevus, 1 melanoma, 2 seborrheic keratosis



In [152]:
def clean_cache():
    import gc
    gc.collect()
    torch.cuda.empty_cache()

In [153]:
# db = pd.read_csv('data/dermoscopyDBtrain.csv', header=0, dtype={'id': str, 'label': int})

# #We show inform
# n = 65
# img_id = db.id[n] 
# label = db.label[n]


# print(f'Image ID: {img_id}')
# print(f'Label: {label}')

Now, let's create a simple function to show an image.




In [154]:
def imshow(image, title_str):
    if len(image.shape)>2:
        plt.imshow(image)
    else:
        plt.imshow(image,cmap=plt.cm.gray)
    plt.title(title_str)        

# plt.figure()
# imshow(io.imread(os.path.join('data/images/', img_id + '.jpg' )),'Image %d'%n)
# plt.figure()
# imshow(io.imread(os.path.join('data/masks/', img_id + '.png')),'Mask %d'%n)

# plt.show()

### Class Dataset

The class `` torch.utils.data.Dataset`` is an abstract class that represents a dataset.

To create our custom dataset in pytorch we must inherit from this class and overwrite the following methods:

- `` __len__`` so that `` len (dataset) `` returns the size of the dataset.
- `` __getitem__`` to support indexing `` dataset [i] `` when referring to sample $i$

We are going to create the train and test datasets of our diagnostic problem. We will read the csv in the initialization method `` __init__`` but we will leave the explicit reading of the images for the method
`` __getitem__``. This approach is more efficient in memory because all the images are not loaded in memory at first, but are read individually when necessary.

Our dataset is going to be a dictionary `` {'image': image, 'mask': mask, 'label': label} ``. You can also take an optional `` transform '' argument so that we can add pre-processing and data augmentation techniques.



We now instantiate the class to iterate over some samples to see what we generate.


In [155]:
# train_dataset = DermoscopyDataset(csv_file='data/dermoscopyDBtrain.csv',
#                                     root_dir='data')

# fig = plt.figure()

# for i in range(len(train_dataset)):
#     sample = train_dataset[i]
#     print(i, sample['image'].shape, sample['label'])

#     ax = plt.subplot(1, 4, i + 1)
#     plt.tight_layout()
#     ax.set_title('Sample #{}'.format(i))
#     ax.axis('off')
#     plt.imshow(sample['image'])

#     if i == 3:
#         plt.show()
#         break

The optional parameter ``maxSize`` in the constructor allows us to subsample the number of images and consequently reduce the size of the dataset. If not set included or maxSize=0, then the dataset will include all the images. This parameter is useful to train in smaller datasets during hyperparameter validation and decision making during design. Working with less images reduces the training time at the expense of obtaining results that may not correlate perfectly to what would happen with the full dataset size. Of course, the larger the trianing dataset, the more stable results but the larger the training time. Hence, it is up to the students the use of this parameter. 

### Transforms
----------

In the previously shown examples we can see that the size of the images is not the same. This would prevent to train a red convolutional neuronal, as the vast majority require fixed-size inputs. Furthermore, the image is not always adjusted to the lesion, and indeed, in some examples lesions are very small compared to the size of the image. It would then be desirable to adjust the input images so that the lesion covers almost the entire image.

To do this, we are going to create some preprocessing code, focusing on 4 transformations:

- `` CropByMask``: to crop the image using the lesion mask
- `` Rescale``: to scale the image
- `` RandomCrop``: to crop the image randomly, it allows us to augment the data samples with random crops
-  ``CenterCrop``: to perform a central crop of the image with the indicated size (useful in test)
-  ``TVCenterCrop``: the same functionality of the previous one but using the method in torchvision. Just an example if you plan to use torchvision transforms.
- `` ToTensor``: to convert numpy matrices into torch tensors (rearranging the axes).

We will define them as callable classes instead of simple functions, as we will not need to pass the transform  parameters every time we call a method. To do this, we only have to implement the `` __call__`` method and, if necessary, the `` __init__`` method.
Then we can use a transformation with the following code:

::

    tsfm = Transform(params)
    transformed_sample = tsfm(sample)


In [156]:
class CropByMask(object):
    """Crop the image using the lesion mask.

    Args:
        border (tuple or int): Border surrounding the mask. We dilate the mask as the skin surrounding 
        the lesion is important for dermatologists.
        If it is a tuple, then it is (bordery,borderx)
    """

    def __init__(self, border):
        assert isinstance(border, (int, tuple))
        if isinstance(border, int):
            self.border = (border,border)
        else:
            self.border = border
            
    def __call__(self, image, mask):

        # h, w = image.size[:2]
        h, w = image.shape[:2]
        #Calculamos los índices del bounding box para hacer el cropping
        sidx=np.nonzero(mask)
        minx=np.maximum(sidx[1].min() - self.border[1], 0)
        maxx=np.minimum(sidx[1].max() + 1 + self.border[1], w)
        miny=np.maximum(sidx[0].min() - self.border[0], 0)
        maxy=np.minimum(sidx[0].max() + 1 + self.border[1], h)
        #Recortamos la imagen
        
        image=image[miny:maxy, minx:maxx,...]

        # image = image.crop([minx, miny, maxx, maxy])

        return image
    
class Rescale(object):
    """Re-scale image to a predefined size.

    Args:
        output_size (tuple or int): The desired size. If it is a tuple, output is the output_size. 
        If it is an int, the smallest dimension will be the output_size
            a we will keep fixed the original aspect ratio.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        h, w = image.shape[:2]

        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        image = transform.resize(image, (int(new_h), int(new_w)))

        return {'image': image, 'label': label}

class RandomCrop(object):
    """Randomly crop the image.

    Args:
        output_size (tuple or int): Crop size. If  int, square crop

    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        if h>new_h:
            top = np.random.randint(0, h - new_h)
        else:
            top=0
            
        if w>new_w: 
            left = np.random.randint(0, w - new_w)
        else:
            left = 0
            
        image = image[top: top + new_h,
                     left: left + new_w]

        return {'image': image, 'label': label}
    
class CenterCrop(object):
    """Crop the central area of the image

    Args:
        output_size (tupla or int): Crop size. If int, square crop

    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        rem_h = h - new_h
        rem_w = w - new_w
        
        if h>new_h:
            top = int(rem_h/2)
        else:
            top=0
            
        if w>new_w: 
            left = int(rem_w/2)
        else:
            left = 0
            
        image = image[top: top + new_h,
                     left: left + new_w]

        return {'image': image, 'label': label}

class TVCenterCrop(object):
    """Crop the central area of the image. Example using the method in torchvision. Requires to
    internally convert from skimage (numpy array) to PIL Image

    Args:
        output_size (tupla or int): Crop size. If int, square crop

    """

    def __init__(self, size):
        self.CC=transforms.CenterCrop(size)

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        pil_image=Image.fromarray(util.img_as_ubyte(image))
        pil_image=self.CC(pil_image)
        image=util.img_as_float(np.asarray(pil_image))
        
        return {'image': image, 'label': label}



class ToTensor(object):
    """Convert ndarrays into pytorch tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # Cambiamos los ejes
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image)

        label=torch.tensor(label, dtype=torch.long)
        
        return {'image': image, 'label': label}
    
class Normalize(object):
    """Normalize data by subtracting means and dividing by standard deviations.

    Args:
        mean_vec: Vector with means. 
        std_vec: Vector with standard deviations.
    """

    def __init__(self, mean,std):
      
        assert len(mean)==len(std),'Length of mean and std vectors is not the same'
        self.mean = np.array(mean)
        self.std = np.array(std)

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        c, h, w = image.shape
        assert c == len(self.mean), 'Length of mean and image is not the same' 
        dtype = image.dtype
        mean = torch.as_tensor(self.mean, dtype=dtype, device=image.device)
        std = torch.as_tensor(self.std, dtype=dtype, device=image.device)
        image.sub_(mean[:, None, None]).div_(std[:, None, None])
    
        return {'image': image, 'label': label}


### Composed Transforms

Now let's apply the different transformations to our images. 

We will rescale the images so that their smallest dimension is 256 and then make random crops of size 224. To compose the transformations ``Rescale`` and ``RandomCrop`` we can use ``torchvision.transforms.Compose``, which is a simple callable class.




In [157]:
class CropByMask2(object):

    def __init__(self, border):
        assert isinstance(border, (int, tuple))
        if isinstance(border, int):
            self.border = (border,border)
        else:
            self.border = border
            
    def __call__(self, image, mask):

        h, w = image.size[:2]

        #Calculamos los índices del bounding box para hacer el cropping
        
        sidx=np.nonzero(mask)
        minx=np.maximum(sidx[1].min() - self.border[1], 0)
        maxx=np.minimum(sidx[1].max() + 1 + self.border[1], w)
        miny=np.maximum(sidx[0].min() - self.border[0], 0)
        maxy=np.minimum(sidx[0].max() + 1 + self.border[1], h)
        #Recortamos la imagen
        
        image = image.crop([minx, maxy, maxx, miny])

        return image

Iterating the dataset
-----------------------------

We can now put everything together to create the train and test datasets with the corresponding transformations.
In summary, every time we sample an image from the dataset (during training):
- We will read the image and the mask
- We will apply the transformations and we will crop the image using a bounding box computed from the mask
- As the final cropping operation is random, we perform data augmentation during sampling

We can easily iterate over the dataset with a ``for i in range`` loop.




Finally, we have to create a dataloader allowing to:

- Sample batches of samples to feed the network during training
- Shuffle data
- Load the data in parallel using multiple cores.

``torch.utils.data.DataLoader`` is an iterator that provides all these features. An important parameter of the iterator is ``collate_fn``. We can specify how samples are organized in batches by choosing the most appropriate function. In any case, the default option should work fine in most cases.




In [158]:
# Auxiliary function to visualize a batch
def show_batch(sample_batched):
    """Mostramos las lesiones de un batch."""
    images_batch, labels_batch = \
            sample_batched['image'], sample_batched['label']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid_border_size = 2
    
    #Generamos el grid
    grid = utils.make_grid(images_batch)
    #Lo pasamos a numpy y lo desnormalizamos
    grid=grid.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    grid = std * grid + mean
    grid = np.clip(grid, 0, 1)
    plt.imshow(grid)
    plt.title('Batch from dataloader')

# for i_batch, sample_batched in enumerate(train_dataloader):
#     print(i_batch, sample_batched['image'].size(),
#           sample_batched['label'])
#     plt.figure()
#     show_batch(sample_batched)
#     plt.axis('off')
#     plt.ioff()
#     plt.show()
        
#     #We show the data of the 3rd batch and stop.
#     if i_batch == 0:
#         break

## Part 2: Fine-tuning a pre-trained model

In the second part of the practice we will build an automatic skin lesion diagnosis system. Instead of training a CNN designed by us from the beginning, we will fine-tune a network that has previously been trained for another task. As seen in the lectures, this usually becomes a good alternative when we do not have many data in the training dataset (in relation to the parameters to be learned).

In particular, we will use the Alexnet CNN, included in the ``torchvision`` package.

### Performance Metric for evaluation
We will start by defining the metric we will use to evaluate our network. In particular, and following the instructions of the organizers of the original ISIC challenge, we will use the area under the ROC or AUC (https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve), but we will calculate 3 different AUCs:
- 1) AUC of binary problem melanoma vs all
- 2) AUC of the binary problem seborrheic keratosis vs all
- 3) AUC average of the previous two

The following function computes AUCs from the complete database outputs:



In [159]:
#Function that computes 2 AUCs: melanoma vs all and keratosis vs all
# scores is nx3: n is the number of samples in the dataset 
# labels is nx1
# Function resturns an array with two elements: the auc values
def computeAUCs(scores,labels):
                
    aucs = np.zeros((2,))
    #Calculamos el AUC melanoma vs all
    scores_mel = scores[:,1]
    labels_mel = (labels == 1).astype(np.int) 
    aucs[0]=metrics.roc_auc_score(labels_mel, scores_mel)

    #Calculamos el AUC queratosis vs all
    scores_sk = scores[:,2]
    labels_sk = (labels == 2).astype(np.int) 
    aucs[1]=metrics.roc_auc_score(labels_sk, scores_sk)
    
    return aucs

### Set-up new dataset

In [160]:
class IsicDataset(Dataset):
    
    def __init__(self, csv_file, img_dir, mask_dir, transform=None, max_size=2000):
        
        self.dataset = pd.read_csv(csv_file, header=0, dtype={'id': str, 'label': int})
        
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        
        self.img_names = self.dataset.id
        self.labels = self.dataset.label

        if max_size > 0:
            idx = np.random.RandomState(seed=42).permutation(range(len(self.dataset)))
            reduced_dataset = self.dataset.iloc[idx[0: max_size]]
            self.dataset = reduced_dataset.reset_index(drop=True)
        
        self.transform = transform
        self.classes = ['nevus', 'melanoma', 'keratosis']

        self.img_files = [f'{img}.jpg' for img in self.img_names]
        self.masks_files = [f'{mask}.png' for mask in self.img_names]        
            
    def __len__(self):
        return len(self.dataset)
        

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.img_dir, self.img_files[idx])       
        mask_path = os.path.join(self.mask_dir, self.masks_files[idx])       
        
        image = Image.open(img_path)
        
        if self.transform:
            if isinstance(self.transform.transforms[0], CropByMask2):
                tr_func = self.transform.transforms[0]
                image = tr_func(image, mask=Image.open(mask_path))
                self.transform.transforms.pop(0)

            image = self.transform(image)
        
        label = self.labels[idx].astype("int64")

        return image, label

In [161]:
class DermoscopyDataset(IsicDataset):
    """Dermoscopy dataset."""

    def __init__(self, *args, **kwargs):
        """
        Args:
            csv_file (string): Path al fichero csv con las anotaciones.
            root_dir (string): Directorio raíz donde encontraremos las carpetas 'images' y 'masks' .
            transform (callable, optional): Transformaciones opcionales a realizar sobre las imágenes.
        """
        super().__init__(*args, **kwargs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.img_dir, self.img_files[idx])       
        mask_path = os.path.join(self.mask_dir, self.masks_files[idx])       
        
        image = io.imread(img_path)
        
        label = self.labels[idx].astype("int64")

        sample = {'image': image, 'label':  label}

        if self.transform:
            if isinstance(self.transform.transforms[0], CropByMask):
                tr_func = self.transform.transforms[0]
                sample['image'] = tr_func(image, mask=io.imread(mask_path))
                self.transform.transforms.pop(0)
            
            sample = self.transform(sample)

        return sample['image'], sample['label']

In [162]:
#Pixel means and stds expected by models in torchvision

def setup_datasets(dataset, transform_compose, max_size=2000):

    #Train Dataset
    train_dataset = dataset(csv_file='data/dermoscopyDBtrain.csv',
                              img_dir='data/images',
                              mask_dir='data/masks',
                              max_size=max_size,
                              transform=transform_compose['train'])
    #Val dataset
    val_dataset = dataset(csv_file='data/dermoscopyDBval.csv',
                              img_dir='data/images',
                              mask_dir='data/masks',
                              transform=transform_compose['val'])

    #Test dataset
    test_dataset = dataset(csv_file='data/dermoscopyDBtest.csv',
                              img_dir='data/images',
                              mask_dir='data/masks',
                              transform=transform_compose['test'])
    
    return train_dataset, val_dataset, test_dataset

### Set-up the data loaders
No we will assign the dataloaders over the training and validation data

In [163]:
#Specify training dataset, with a batch size of 8, shuffle the samples, and parallelize with 4 workers

def setup_dataloaders(dataset, transform, num_workers, batch_sizes, max_train_size):
    
    train_dataset, val_dataset, test_dataset = setup_datasets(dataset, transform, max_train_size)
        
    train_dataloader = DataLoader(train_dataset, batch_size=batch_sizes['train'], shuffle=True, num_workers=num_workers)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_sizes['val'], shuffle=False, num_workers=num_workers)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_sizes['test'], shuffle=False, num_workers=num_workers)
    
    
    datasets = {'train' : train_dataset, 'val': val_dataset, 'test': test_dataset}
    dataloaders = {'train' : train_dataloader, 'val': val_dataloader, 'test': test_dataloader}
    dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}  # 'test': len(test_dataset)
    
    return datasets, dataloaders, dataset_sizes

### Training function

We continue defining the function to train our classifier:

In [164]:
#train_model parameters are the network (model), the criterion (loss),
# the optimizer, a learning scheduler (una estrategia de lr strategy), and the training epochs
def train_model(model, criterion, optimizer, scheduler, num_epochs,
                dataset_sizes, dataloaders, device):
    
    since = time.time()

    num_classes = 3

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc, best_auc = 0, 0
    best_epoch = -1
    
    #Loop of epochs (each iteration involves train and val datasets)
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Cada época tiene entrenamiento y validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set the model in training mode
            else:
                model.eval()   # Set the model in val mode (no grads)
            
            num_samples = dataset_sizes[phase]
            
            # Create variables to store outputs and labels
            outputs_m = np.zeros((num_samples, num_classes), dtype=np.float)
            labels_m = np.zeros((num_samples, ), dtype=np.int)

            running_loss = 0.0
            running_corrects = 0

            cont_samples = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device).float()
                labels = labels.to(device)

                batch_size = labels.shape[0]

                optimizer.zero_grad()  # Set grads to zero

                # Forward
                # Register ops only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward & parameters update only in train
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # Accumulate the running loss
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                outputs = F.softmax(outputs.data, dim=1)
                # Store outputs and labels 
                outputs_m[cont_samples: cont_samples + batch_size, ...] = outputs.cpu().numpy()
                labels_m [cont_samples: cont_samples + batch_size]      = labels.cpu().numpy()
                cont_samples += batch_size

            #At the end of an epoch, update the lr scheduler    
            if phase == 'train':
                scheduler.step()
            
            #Accumulated loss by epoch
            epoch_loss = running_loss / num_samples
            epoch_acc = running_corrects / num_samples

            epoch_auc = computeAUCs(outputs_m, labels_m)
            
            print(f'{phase} -> Loss: {epoch_loss}, Avg. acc: {epoch_acc}\
                  Mel: {epoch_auc[0]} / Seb: {epoch_auc[1]}, Avg. AUC: {epoch_auc.mean()}')

            # Deep copy of the best model
            if phase == 'val' and epoch_acc > best_auc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_epoch = epoch
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best model in epoch {:d} avg {:4f}'.format(best_epoch, best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Fine-tuning of a pre-trained CNN
Once we have defined the training and evaluation functions, we will fine-tune AlexNet CNN using our database. In addition, we define the loss, the optimizer and the lr scheduler:

## Part 3: Evaluation (Important)
The evaluation of this practice will be done through a challenge. For this, students are asked to provide the following:
- Two submissions for the test set scores (unnormalized), each one represented by a 600x3 matrix where 600 is the number of test samples, and 3 are the classes considered in the problem. The matrix must be provided in csv format (with 3 numbers per row separated by ',').

In addition, students will submit a short report (1 side at most for the description, plus 1 side for references and 1 for figures, if necessary) where they will describe the most important aspects of the proposed solution and include a table with the validation results achieved by their extensions/decisions. The objective of this report is for the teacher to assess the developments / extensions / decisions made by the students when optimizing their system. And the table is asked to demonstrate that, at least in validation, their decisions helped to improve the system performance.  You don't need to provide an absolute level of detail about the changes made, just list them, briefly discuss their purpose and show their impact in the table.

The deadline for delivery of the results file and the report is Wednesday Nov 3 at 23:59.



Next we provide some functions that allow to test the network and create the csv file with the outputs.


In [165]:
### Code that generates the test matrix

def test_model(model, dataset, dataloader, device):
    model.eval()  # Ponemos el modelo en modo evaluación

    num_samples, num_classes = len(dataset), len(dataset.classes)  # Tamaño del dataset

    # Creamos las variables que almacenarán las salidas y las etiquetas
    outputs_m = np.zeros((num_samples, num_classes), dtype=np.float)
    cont_samples = 0

    # Iteramos sobre los datos
    for inputs, _labels in dataloader:
        inputs = inputs.to(device)

        batch_size = inputs.shape[0]  # Tamaño del batch

        with torch.torch.no_grad():  # Paso forward
            outputs = model(inputs)
            outputs = F.softmax(outputs.data, dim=1)  # Aplicamos un softmax a la salida
            outputs_m[cont_samples: cont_samples + batch_size, ...] = outputs.cpu().numpy()
            cont_samples += batch_size

    return outputs_m

Running the previous function and obtaining the matrix with the scores (Nx3)

In [166]:
# outputs=test_model(model_ft)
# print(outputs)

And finally save the matrix into a csv file


In [169]:
class BaseNet(object):

    def __init__(self, model, transform_c, max_train_size, criterion, optimizer, scheduler, num_workers, batch_sizes,
                 device):

        self.datasets, self.dataloaders, self.sizes = self._setup(transform_c, num_workers, batch_sizes, max_train_size)
        self.num_classes = len(self.datasets['train'].classes)

        self.model = model(pretrained=True)
        self.model_name = model.__name__.lower()
        self.trained_model = None
        self.model_path = None

        self.criterion = criterion

        optimizer_func = optimizer['func']
        self.optimizer = optimizer_func(self.model.parameters(), **optimizer['args'])

        scheduler_func = scheduler['func']
        self.scheduler = scheduler_func(self.optimizer, **scheduler['args'])

        self._adjust_network()

        self.device = device
        self.model.to(self.device)

    @staticmethod
    def _setup(transform_c, num_workers, batch_sizes, max_train_size):
        return setup_dataloaders(transform_c['dataset'], transform_c, num_workers, batch_sizes, max_train_size)

    def _adjust_network(self):

        if isinstance(self.model, models.AlexNet) or isinstance(self.model, models.VGG): # or \
                # isinstance(self.model, models.EfficientNet):
            num_features = self.model.classifier[-1].in_features
            self.model.classifier[-1] = nn.Linear(num_features, self.num_classes)

        elif isinstance(self.model, models.ResNet) or isinstance(self.model, models.ShuffleNetV2) or \
                isinstance(self.model, models.Inception3) or isinstance(self.model, models.GoogLeNet):
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Linear(num_ftrs, self.num_classes)

        elif isinstance(self.model, dmodels.MaskRCNN):
            self.model.roi_heads.mask_predictor = dmodels.mask_rcnn.MaskRCNNPredictor(256, 256, self.num_classes)

        elif isinstance(self.model, dmodels.FasterRCNN):
            in_features = self.model.roi_heads.box_predictor.cls_score.in_features
            self.model.roi_heads.box_predictor = dmodels.faster_rcnn.FastRCNNPredictor(in_features, self.num_classes)

    def trainloop(self, num_epochs=25, path='/content/drive/MyDrive'):
        torch.cuda.empty_cache()

        self.trained_model = train_model(
            self.model, self.criterion, self.optimizer, self.scheduler,
            num_epochs=num_epochs,
            dataloaders={'train': self.dataloaders['train'], 'val': self.dataloaders['val']},
            dataset_sizes=self.sizes,
            device=self.device
        )

        self.model_path = f'{path}/{self.model_name}_model_n{num_epochs}.pt'
        self.save_model(self.model_path)

        return self.trained_model

    def evaluate(self, output_path, dataset='test', model_path=None, save_to_csv=False):
        clean_cache()
        if model_path is None:
            model_path = self.model_path

        # model = self.load_model(model_path)
        model = self.trained_model

        outputs = test_model(model, self.datasets[dataset], self.dataloaders[dataset], device=self.device)

        if save_to_csv:
            self.save_output_to_csv(outputs, path=f'{output_path}/{self.model_name}_output_{dataset}.csv')

        return outputs

    def save_model(self, path):
        model_scripted = torch.jit.script(self.trained_model)
        model_scripted.save(path)

    @staticmethod
    def load_model(model):
        return torch.jit.load(model)

    @staticmethod
    def save_output_to_csv(outputs, path):
        import csv
        with open(path, mode='w', newline='') as out_file:
            csv_writer = csv.writer(out_file, delimiter=',')
            csv_writer.writerows(outputs)

In [170]:
def get_data_transforms(mode='torch'):
    original_c = transforms.Compose([
            CropByMask(15),
            Rescale(256),
            CenterCrop(224),
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
    )
    train_composed = transforms.Compose([
            CropByMask2((25, 25)),
            transforms.CenterCrop(224),
            transforms.Resize((256, 256)),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
    )
    test_composed = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
    )
    if mode == 'torch':
      return {'train': train_composed, 'val': train_composed, 'test': test_composed, 
              'dataset': IsicDataset}
    
    elif mode == 'original':
      return {'train': original_c, 'val': original_c, 'test': original_c,
              'dataset': DermoscopyDataset}

def gen_model(params, device):
    return BaseNet(
        model=params.get('model'),
        transform_c=params.get('transform'),
        max_train_size=params.get('max_train_size'),
        criterion=params.get('criterion'),
        optimizer=params.get('optimizer'),
        scheduler=params.get('scheduler'),
        num_workers=params.get('num_workers'),
        batch_sizes=params.get('batch_sizes'),
        device=device
    )

In [171]:
def gen_model_params(model, data_transforms, max_train_size):
    return {
        'model': model,
        'transform': data_transforms,
        'max_train_size': max_train_size,
        'criterion': nn.CrossEntropyLoss(),
        'optimizer':
            {'func': optim.SGD,
             'args': {'lr': 0.001, 'momentum': 0.9}},
        'scheduler':
            {'func': lr_scheduler.StepLR,
             'args': {'step_size': 7, 'gamma': 0.1}},
        'num_workers': 8,
        'batch_sizes': {
            'train': 64,
            'val': 128,
            'test': 128
        }
    }

In [172]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_transforms = get_data_transforms(mode='torch')

detection_models = [dmodels.fasterrcnn_resnet50_fpn,
                    dmodels.maskrcnn_resnet50_fpn]

test_models = [
      # models.vgg19_bn,
      # models.vgg16_bn,
    # models.efficientnet_b7,
    # models.resnext101_32x8d,
    # models.resnext50_32x4d,

    # models.alexnet,
    # models.resnet18,
    # models.resnet50,
    models.resnet101,
    # models.resnext50_32x4d,
    # models.resnext101_32x8d,
    # models.googlenet,
    # models.vgg19_bn,
    # models.vgg19,
    # models.shufflenet_v2_x0_5
]

model_params = [gen_model_params(model, data_transforms, max_train_size=2000) for model in test_models]
print(device)

cuda:0


In [ ]:
net_models = [gen_model(params, device) for params in model_params]

for model in net_models:
    model_trained = model.trainloop(num_epochs=25)
    outputs = model.evaluate(output_path='/content/drive/MyDrive', save_to_csv=True)

Epoch 1/25
----------
train -> Loss: 1.2118875522613526, Avg. acc: 0.23650000989437103                  Mel: 0.5785991014990364 / Seb: 0.6181823921494349, Avg. AUC: 0.5983907468242357
val -> Loss: 1.0478185844421386, Avg. acc: 0.4866666793823242                  Mel: 0.6916666666666667 / Seb: 0.7458112874779541, Avg. AUC: 0.7187389770723104

Epoch 2/25
----------
train -> Loss: 1.071602681159973, Avg. acc: 0.45250001549720764                  Mel: 0.7207970742809032 / Seb: 0.8194703754814153, Avg. AUC: 0.7701337248811593
val -> Loss: 1.0135192569096882, Avg. acc: 0.54666668176651                  Mel: 0.7575000000000001 / Seb: 0.8247354497354498, Avg. AUC: 0.7911177248677249

Epoch 3/25
----------
train -> Loss: 0.9145900378227234, Avg. acc: 0.6330000162124634                  Mel: 0.7894787905098303 / Seb: 0.8848571763581099, Avg. AUC: 0.83716798343397
val -> Loss: 0.8883570194244385, Avg. acc: 0.6600000262260437                  Mel: 0.7444444444444445 / Seb: 0.9186507936507937, Avg.

In [ ]:
outputs = model.evaluate(output_path='/content/drive/MyDrive', save_to_csv=True)